In [23]:
#加载mnist数据集
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score,confusion_matrix
import pandas as pd
from tqdm import tqdm

In [16]:
def read_file(file_name):
    with open(file_name,'r',encoding='utf-8') as f:
        data = f.readlines()
    label_map = {}
    label_map["ham"] = -1
    label_map["spam"] = 1
    
    
    text = list()
    label = list()
    label_num = list()
    for item in data:
        item = item.strip()
        item = item.split(" ")
        item_len = len(item)
        tmp_list = list()
        if(item[1] in ["ham","spam"]):
            label.append(item[1])
            label_num.append(label_map[item[1]])
            for index in range(2,item_len,2):
                tmp_list.append(item[index])
        text.append(" ".join(tmp_list))
    return text,label,label_num

    

In [17]:
train_text,train_label,train_label_num = read_file("../data/train")
test_text,test_label,test_label_num = read_file("../data/test")

In [18]:
train_text[0]

'need fw 35 39 thanks thread 40 copy else correlator under companies 25 he 26 168 29 content 1 6 5 4 review we john 17 use 15 20 classes may a back l 01 produced i yes 10 713 v6 p original doc 2001 comments x week to harry 110 smtpsvc has who sender would any jan be index text intended and urn email cc 1600 can re corp fri response 65 plain confidential 853 you mailto anything am our they for info of are on exchange topic information transfer or msmbx01v 4418 questions now them pm mime binary subject tnef nahou version thank encoding contracts me tuesday ena trading just let ms return 0500 attach attached account mimeole but send individual him type 192 2195 sent enron please when contract class here msmbx05v company this call have june currently while if path find with is it error your agreement the in know their energy'

In [19]:
#提取TF-IDF特征
cv = CountVectorizer()
part_fit = cv.fit(train_text) # 以部分句子为参考
train_all_count = cv.transform(train_text) # 对训练集所有邮件统计单词个数
test_all_count = cv.transform(test_text) # 对测试集所有邮件统计单词个数
tfidf = TfidfTransformer()
train_tfidf_matrix = tfidf.fit_transform(train_all_count)
test_tfidf_matrix = tfidf.fit_transform(test_all_count)

print('训练集', train_tfidf_matrix.shape)
print('测试集', test_tfidf_matrix.shape)

训练集 (9000, 965)
测试集 (1000, 965)


In [21]:
#获取w 和 b 
def get_parameter(x,y,iter_num=50,lr=0.0001):
    #初始化w和b
    m,n = x.shape[0],x.shape[1]
    w = np.zeros((n,1))
    b = 0
    for i in tqdm(range(iter_num)):
        #取每一条数据并判断是否为假的
        for index in range(m):
            single_x = x[index]
            single_x = single_x.reshape((1,n))
            single_y = y[index]
            
            adjust_data = -1*single_y * (single_x.dot(w)+b)
            if adjust_data[0]>=0:
                t = lr*(single_x*single_y)
                t = t.reshape((n,1))
                w = w + t
                b = b + lr*single_y
    return w,b

In [24]:
w,b = get_parameter(train_tfidf_matrix,train_label_num)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [02:38<00:00,  3.17s/it]


In [28]:
#测试数据
def test_accuracy(x,y,w,b):
    error_num = 0
    for index in range(x.shape[0]):
        single_x = x[index]
        single_y = y[index]
        adjust_data = -1*single_y * (single_x.dot(w)+b)
        if adjust_data[0]>=0:
            error_num += 1
    print("accuracy: ",1-error_num/x.shape[0])

In [29]:
test_accuracy(test_tfidf_matrix,test_label_num,w,b)

accuracy:  0.991
